# Delay Calculation Validation Experiments

- Note that you'll have to be on an updated branch ... (to be pushed pending exising PRs)
     - There is way to much going on in the current PRs, don't want to add complexity just yet

In [2]:
import os, os.path as op
import numpy as np
import xarray as xr
from datetime import datetime
from RAiDER.llreader import BoundingBox
from RAiDER.losreader import Zenith, Conventional, Raytracing
from RAiDER.delay import tropo_delay
%pdb off

# change this to the directory with the notebook
WD = op.join(op.expanduser('~'), 'Software_InSAR','RAiDER-docs_git','notebooks', 'Synthetic_Test')
os.chdir(WD)
print ('Working directory (WD):', WD)

Automatic pdb calling has been turned OFF
Working directory (WD): /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/Synthetic_Test


### helper functions

update_wm is the one to edit

In [33]:
def update_wm(wm_file):
    """ Edit the values in a weather model file 

    write it out with extension _SYNTH
    Return the file name for input into tropo_delay

    wet_total/hydro_total used for zenith/proj
    wet/hydro used for ray
    """

    with xr.open_dataset(wm_file) as ds:
        # edit t, p, e, wet, hydro, wet_total, hydro_total
        ds['wet']   = ('z y x'.split(), np.zeros_like(ds['wet']))
        ds['hydro'] = ('z y x'.split(), np.zeros_like(ds['hydro']))
        
        ds['wet_total']   = ('z y x'.split(), np.zeros_like(ds['wet_total']))
        ds['hydro_total'] = ('z y x'.split(), np.zeros_like(ds['hydro_total']))
        
    print (ds)
    dst = f'{op.splitext(wm_file)[0]}_SYNTH.nc'
    ds.to_netcdf(dst)
    print ('Wrote synthetic weather model file to:', dst)
    return dst


In [4]:
def compare_golden(wd, ext='ray'):
    """ Quick check if the synth result is different than real weather wm """
    ds_orig = xr.open_dataset(f'{wd}/golden_data/GMAO_tropo_20181113T230000_{ext}.nc')
    da_dry0 = ds_orig['hydro']
    ds_new  = xr.open_dataset(f'{wd}/Synthetic_tropo_20181113T110000_{ext}.nc')
    da_dry1 = ds_new['hydro']
    equal = np.allclose(da_dry0.data, da_dry1.data, equal_nan=True)
    print ('Orig == synthetic?', equal)
    return

### Get/Set defaults
- Mimic test_scenario_3.py

In [22]:
dt  = datetime(2018, 11, 13, 23)
dts = dt.strftime('%Y_%m_%d_T%H_%M_%S')

height_levels = np.array([0., 100., 500., 1000.])
aoi = BoundingBox([36.8, 36.85, -76.15, -76.05])
aoi.add_buffer(buffer=1.)

# get redownload (with change to name) with !eof -m S1A -d {dts.replace('_', '')}
orbit = f'{WD}/S1A_OPER_AUX_POEORB_OPOD_20181203T120749_V20181112T225942_20181114T005942.EOF'

In [23]:
los = Zenith()
ext = 'ztd'

# los = Conventional(orbit)
# ext = 'std'

# los = Raytracing(orbit, time=dt)
# ext = 'ray'

In [29]:
# original weather model file 
# can be redownloaded with !raider.py raider_example_3.yaml
wm0 = f'{WD}/weather_files/GMAO_{dts}_36N_39N_78W_75W.nc'
wm  = update_wm(wm0)

<xarray.Dataset>
Dimensions:      (x: 12, y: 13, z: 145)
Coordinates:
  * x            (x) float64 -77.81 -77.5 -77.19 -76.88 ... -75.0 -74.69 -74.38
  * y            (y) float64 35.25 35.5 35.75 36.0 ... 37.5 37.75 38.0 38.25
  * z            (z) float32 -500.0 -300.0 -200.0 ... 7.458e+04 8.03e+04
Data variables:
    WGS84        |S1 ...
    latitude     (z, y, x) float64 ...
    longitude    (z, y, x) float64 ...
    t            (z, y, x) float32 ...
    p            (z, y, x) float32 ...
    e            (z, y, x) float32 ...
    wet          (z, y, x) float32 ...
    hydro        (z, y, x) float32 ...
    wet_total    (z, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    hydro_total  (z, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    Conventions:   CF-1.6
    datetime:      2018_11_13T23_00_00
    date_created:  2023_03_08T17_52_47
    title:         Weather model data and delay calculations
Wrote synthetic weather model file to: /Users/buzza

### Compute Delays

In [20]:
ds  = tropo_delay(dt, wm, aoi, los, height_levels, cube_spacing_m=5000)[0]

Output SNWE: [36.75, 36.85, -76.15, -76.0]
Output cube spacing: 0.05
Processing slice 1 / 4: 0.0
Processing slice 2 / 4: 100.0
Processing slice 3 / 4: 500.0
Processing slice 4 / 4: 1000.0


In [9]:
out = f'{WD}/Synthetic_tropo_20181113T110000_{ext}.nc'
ds.to_netcdf(out, mode="w")
print ('Wrote:', out)

Wrote: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/Synthetic_Test/Synthetic_tropo_20181113T110000_ray.nc


### Check against analytical solutions

In [10]:
compare_golden(WD)

Orig == synthetic? True


In [35]:
from RAiDER.models.gmao import GMAO
Obj = GMAO()

In [31]:
## zenith delays in weatherModel.py:

class GMAOdummy(object):
    def __init__(self):
        self._k1 = 0.776  # [K/Pa]
        self._k2 = 0.233  # [K/Pa]
        self._k3 = 3.75e3  # [K^2/Pa]
        self._p  = np.ones([2,3,4])
        self._e  = np.ones([2,3,4])
        self._t  = np.ones([2,3,4])
        self._zs = np.array([-500., -300., -200., -100.,  -50.]) # model levels
        self._get_wet_refractivity()
        self._get_hydro_refractivity()

    def _get_wet_refractivity(self):
        '''
        Calculate the wet delay from pressure, temperature, and e
        '''
        self._wet_refractivity = self._k2 * self._e / self._t + self._k3 * self._e / self._t**2

    def _get_hydro_refractivity(self):
        '''
        Calculate the hydrostatic delay from pressure and temperature
        '''
        self._hydrostatic_refractivity = self._k1 * self._p / self._t

    
    def getWetRefractivity(self):
        return self._wet_refractivity

   
    def getHydroRefractivity(self):
        return self._hydrostatic_refractivity

   
    def _getZTD(self, zref=None):
        wet   = self.getWetRefractivity()
        hydro = self.getHydroRefractivity()


        # Get the integrated ZTD
        wet_total, hydro_total = np.zeros(wet.shape), np.zeros(hydro.shape)
        for level in range(wet.shape[2]):
            wet_total[..., level] = 1e-6 * np.trapz(
                wet[..., level:], x=self._zs[level:], axis=2
            )
            hydro_total[..., level] = 1e-6 * np.trapz(
                hydro[..., level:], x=self._zs[level:], axis=2
            )
        self._hydrostatic_ztd = hydro_total
        self._wet_ztd = wet_total
        return

Obj = GMAOdummy()
# Obj._getZTD()

## Questions

- is our hydrostatic equation missing a term?
ours: $$0.776 \frac{P_d}{T}$$
[Li et al, Eq. 2](https://amt.copernicus.org/articles/14/6379/2021/): $$0.776  \frac{P_d}{T} + 0.776 \frac{M_w}{M_d}\frac{P_w}{T}$$

where: $P_d$ (in hPa) is the partial pressure of the dry constituent, $P_w$ (in hPa) is the partial pressure of water vapor, T (in K) is the partial temperature. They don't define $M_w$ and $M_d$/

Meanwhile, from [Stephens et al., Eq. 4](https://www.mdpi.com/2072-4292/12/5/782)(also Jolivet): $$0.776  \frac{R_d}{9.8}(P)$$

where $R_d$ is the specific gas constants for dry air 

Agreement with [Bekaert et al., 2015](https://www.sciencedirect.com/science/article/pii/S0034425715301231):
$$k_1\left(\frac{P}{T}\right)_{hyd} + \left(k_2\frac{e}{T} + k_3\frac{e}{T}^2\right)_{wet}$$


I suppose we ignore the $P_w$ wrt Li et al., but why are we so different from Jolivet?
